# Import data from Google Cloud Storage (GCS)

It's convenient and secure to host data in the cloud for data labeling, then sync task references to Label Studio to allow data annotators to view and label the tasks without your data leaving the secure cloud bucket. 

If your data is hosted in Google Cloud Storage (GCS), you can write a Python script to continuously sync data from the bucket with Label Studio. Follow this example to see how to do that with the [Label Studio SDK](https://api.labelstud.io/). 

## Get your GCS bucket ready

Connect to your GCS bucket and create a list of task references that Label Studio can use, based on the contents of your bucket. You need to ensure you have [authenticated to the Google Cloud using GOOGLE_APPLICATION_CREDENTIALS](https://developers.google.com/workspace/guides/auth-overview)

In [1]:
import os

BUCKET_NAME = 'your-bucket-name'  # specify your bucket name here
PREFIX = 'sub/directory'  # specify your prefix here
# assuming you service account key is stored in GOOGLE_APPLICATION_CREDENTIALS
GOOGLE_APPLICATION_CREDENTIALS_FILE = os.getenv('GOOGLE_APPLICATION_CREDENTIALS') 
with open(GOOGLE_APPLICATION_CREDENTIALS_FILE) as f:
    credentials = f.read()

## Create a Label Studio Project

Connect to the Label Studio API with your personal API key, which you can retrieve from your user account page, and confirm you can successfully connect:

In [2]:
from label_studio_sdk.client import LabelStudio
LABEL_STUDIO_URL = 'http://localhost:8080'
API_KEY = '27c982caa9e599c9d81b25b00663e7d4f82c9e3c'

ls = LabelStudio(base_url=LABEL_STUDIO_URL, api_key=API_KEY)

Create the project. In this example, the project is a basic [image object detection project](https://labelstud.io/templates/image_bbox.html):

In [11]:
project = ls.projects.create(
    title='Using Images from GCS',
    label_config='''
    <View>
        <Image name="image" value="$image"/>
        <RectangleLabels name="objects" toName="image">
            <Label value="Airplane"/>
            <Label value="Car"/>
        </RectangleLabels>
    </View>
    '''
)

## Connect to your GCS bucket

The following parameters are required to connect to your GCS bucket:

In [13]:
connection_params = {
    # project id where the storage is connected
    'project': project.id,
    # bucket name
    'bucket': BUCKET_NAME,
    # subdirectory in the bucket
    'prefix': PREFIX,
    # regex filter for files in the bucket - when not used, no files are retrieved
    'regex_filter': '.*\.jpg',
    # google application credentials
    'google_application_credentials': credentials,
    # interpret the file as a blob url, like images 'gs://bucket_name/prefix/filename.jpg'
    'use_blob_urls': True,
    # presign the urls to expose data with `https://` protocol
    'presign': True,
    # presign ttl in minutes
    'presign_ttl': 1
}

Before creating the connection, validate the connection parameters to ensure they are correct:

In [14]:
ls.import_storage.gcs.validate(**connection_params)

Connect your newly-created project to your GCS bucket:

In [15]:
storage = ls.import_storage.gcs.create(**connection_params)

## Sync tasks from GCS to Label Studio

After connecting to your bucket, you can import your private GCS links to Label Studio. When opening in Label Studio interface, they're automatically presigned for security! 

In [16]:
ls.import_storage.gcs.sync(id=storage.id)

GcsImportStorage(id=7, type='gcs', synchronizable=True, presign=True, bucket='heartex-test-images', prefix='cells/', regex_filter='.*\\.jpg', use_blob_urls=True, google_application_credentials=None, google_project_id=None, last_sync=datetime.datetime(2024, 6, 10, 14, 8, 10, 127188, tzinfo=datetime.timezone.utc), last_sync_count=132, last_sync_job=None, status='completed', traceback=None, meta={'attempts': 1, 'time_queued': '2024-06-10 14:08:06.561108+00:00', 'time_in_progress': '2024-06-10 14:08:06.566219+00:00', 'time_last_ping': '2024-06-10 14:08:06.566219+00:00', 'time_completed': '2024-06-10 14:08:10.127198+00:00', 'duration': 3.560979, 'tasks_existed': 0}, title=None, description=None, created_at=datetime.datetime(2024, 6, 10, 14, 7, 58, 472871, tzinfo=datetime.timezone.utc), presign_ttl=1, project=67)

## Conclusion

In a few lines of code you assessed the data in your bucket, set up a new labeling project, and synced the tasks to the project. You can adapt this example to more easily create a data creation to data labeling pipeline.